<a href="https://colab.research.google.com/github/pulkitv/test-rag/blob/main/Audio_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openai==0.28
# !pip install pyannote.audio
!pip install openai-whisper

# !pip install torch==1.10.0+cu102
# !pip install pyannote.audio==0.0.1

!pip install git+https://github.com/openai/whisper.git
# !pip install gtts
# !pip install pydub
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 

In [4]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
# ACTIVELOOP_TOKEN = userdata.get('ACTIVELOOP_TOKEN')
# LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
# HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')
GMAIL_PASSWORD = userdata.get('GMAIL_PASSWORD')


# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["GMAIL_PASSWORD"] = GMAIL_PASSWORD

print("Done")

Done


In [8]:
from google.colab import files
import shutil

# Upload audio file
uploaded = files.upload()

# Move the uploaded file to the working directory
audio_file_path = list(uploaded.keys())[0]


Saving Meeting 1.mp3 to Meeting 1.mp3


In [10]:
import whisper
# from pyannote.audio.pipelines import SpeakerDiarization
# from pyannote.core import Segment
import torch

model = whisper.load_model("base")

# Perform transcription with Whisper and get word-level timestamps
result = model.transcribe(audio_file_path, word_timestamps=True, task='translate')

print(result)

torch.cuda.empty_cache()



100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 72.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:176: UserWarning: Word-level timestamps on translations may not be reliable.
  warnings.warn("Word-level timestamps on translations may not be reliable.")


{'text': " And further we can file those documents after like here's go in the consent there. And if like the user is not having a existing account on DG locker, so like here you go for the sign up page and there are long process of like loading the docs on the like on the docs on the DG locker and then again you will give the consent of like whatever documents he is like likely to provide those room cards further we can call away their files as document. Am I right? Exactly. So I just wanted to one thing, there's one thing, like here if like in the DG locker you have an option to open multiple documents like a driving and some app and card and what I was saying, Adhalka, 10th mark, 10th mark, 10th mark, okay and many more documents you can upload on the app itself. So like ARC use case we only have two to three that document they put in for us like Adhalka, Devin license or in any case fan card okay. So like can we have do we have an option that while he is signing up so only those do

In [15]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Example transcription (replace this with your actual transcription)
transcribed_text = result['text']

# Split the text into smaller chunks (tokens or paragraphs)
def split_text(text, max_chunk_size=4000):
    words = text.split()
    chunks = []
    current_chunk = []
    current_size = 0

    for word in words:
        current_size += len(word) + 1
        if current_size > max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_size = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

chunks = split_text(transcribed_text)

# Use a Sentence-BERT model or OpenAI embeddings to convert text to vectors
model = SentenceTransformer('all-MiniLM-L6-v2')
vectors = model.encode(chunks)

# Store vectors in a FAISS index for similarity search
dim = len(vectors[0])  # Vector dimensions
index = faiss.IndexFlatL2(dim)  # L2 distance metric
index.add(np.array(vectors))  # Add vectors to the FAISS index

# Now the vectors are indexed in FAISS
print("Vectors stored in FAISS.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vectors stored in FAISS.


In [17]:
# Function to retrieve relevant chunks based on a query
def retrieve_relevant_chunks(query, index, top_k=3):
    query_vector = model.encode([query])  # Convert query to vector
    _, indices = index.search(np.array(query_vector), top_k)  # Search for similar chunks

    relevant_chunks = [chunks[i] for i in indices[0]]
    return relevant_chunks

# Example query to retrieve relevant chunks
query = "You are a personal AI assistant who is hired to take notes and summarize meetings for future reference. Assume that the transcription provided is of a business meeting."
relevant_chunks = retrieve_relevant_chunks(query, index)

# Print relevant chunks
for chunk in relevant_chunks:
    print(chunk)


on you. Wow. Then your phone number is the email ID there and this is your summary. It's summary miss, right? You'll get your score. This is your score. Okay. Okay. I had to show you this because I don't have the address so the just the score. But you would want just the score. Yeah. Where the input would be mobile number name response would be just the score. Name is also. It's what I look something name also has to be correct or the name being incorrect and like the spelling of the name being correct or something. Yeah, this depends upon case to game business. Okay. All the name are that these both are the mandatory cases. Monday parameters. These are both. Yeah, these are my. Okay. The name and mobile number. Phone number we have from the guest initially because he logs in to zoom car and then we can go forward. But my name. I can go on both. If you don't have the name. There's another API. Yeah, just type your mobile on. I'll check the alphabet from post. I think that number. Now t

In [21]:
import openai

# Set OpenAI API key
openai.api_key = OPENAI_API_KEY

# Function to summarize text using GPT-3.5 or GPT-4
def summarize_text(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize in bullet points (within 500 tokens) the following conversation (from an audio recording) without missing any important details. Segregate if there are multiple topics discussed. In each topic, go one level deeper - what was the speaker actually trying to say. Include any action items that need to be done. Do not include any follow up remarks/questions: {text}"}
        ],
        max_tokens=500
    )
    return response.choices[0].message['content'].strip()

# Summarize the retrieved relevant chunks
final_summary = summarize_text("\n\n".join(relevant_chunks))
print("Final Summary: ", final_summary)


Final Summary:  - The conversation started with discussing a scoring system based on mobile numbers and names for a certain application.
  - The speaker mentioned that the name and mobile number are mandatory inputs for the scoring.
  - They talked about using an API to verify bank account details through a penny drop process.
  - An example was given about how companies can use phone numbers to obtain background details and verify bank accounts.
  - The last step involved customers completing a digital offer with minimal input required.
  - Testing and access were planned as next steps, with commercial details to be shared by one of the parties.

- Regarding live data testing vs. staging:
  - The importance of testing certain sensitive APIs on live data, like the credit score API, was discussed.
  - The need for a static IP address for white-listing to conduct live data testing was highlighted.
  - Testing with simulation data was suggested for less sensitive APIs to ensure proper fun

In [22]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

# Function to send email
def send_email(conversation, summary):
    # Set up the server
    smtp_server = "smtp.gmail.com"  # Using Gmail's SMTP server
    smtp_port = 587  # Standard SMTP port for TLS
    sender_email = "vashishta.pulkit@gmail.com"  # Your Gmail email address
    receiver_email = "vashishta.pulkit@gmail.com"  # Receiver's email address
    email_password = GMAIL_PASSWORD  # Your Gmail App Password (generated from Google account)

    # Create the email content
    subject = "Meeting Summary and Conversation"
    body = f"""
    <h2>Conversation:</h2>
    <pre>{conversation}</pre>

    <h2>Summary:</h2>
    <pre>{summary}</pre>
    """

    # Create MIME message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body to the email
    msg.attach(MIMEText(body, "html"))

    # Connect to the server and send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, email_password)  # Log in to your email account
            server.sendmail(sender_email, receiver_email, msg.as_string())  # Send the email
            print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")

# Send the email
send_email(chunk, final_summary)


Email sent successfully!
